# PUC Minas 
## Trabalho de Conclusão de Curso - <b>Inteligência Artificial e Aprendizado de Máquina</b>


- Autor: <b>Bruno Defante da Silva</b>

- Tema: <b>Modelo preditivo para inferência em paradas cardíacas</b>

- Conjunto de Dados: [Heart Failure Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction)


Este notebook entra como parte fundamental na construção do trabalho de conclusão de curso, pois nele será desenvolvida toda a parte prática do trabalho.

O Projeto será desenvolvido utilizando a base de dados extraída do _Kaggle_ e poderá ser acessada no link disponibilizado acima.

<p> Aqui, serão importadas as biblíotecas necessárias:</p>

In [108]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

### Ingestão dos Dados

In [4]:
df = pd.read_csv('../data/heart.csv')

<b>Este tratamento serve apenas para deixar os nomes das colunas em caixa alta para facilitar o manuseio</b>

In [16]:
df.columns = [col.upper() for col in df.columns]

# Análise Univariada

In [129]:
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AGE             918 non-null    int64  
 1   SEX             918 non-null    object 
 2   CHESTPAINTYPE   918 non-null    object 
 3   RESTINGBP       918 non-null    int64  
 4   CHOLESTEROL     918 non-null    int64  
 5   FASTINGBS       918 non-null    int64  
 6   RESTINGECG      918 non-null    object 
 7   MAXHR           918 non-null    int64  
 8   EXERCISEANGINA  918 non-null    object 
 9   OLDPEAK         918 non-null    float64
 10  ST_SLOPE        918 non-null    object 
 11  HEARTDISEASE    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


,AGE,SEX,CHESTPAINTYPE,RESTINGBP,CHOLESTEROL,FASTINGBS,RESTINGECG,MAXHR,EXERCISEANGINA,OLDPEAK,ST_SLOPE,HEARTDISEASE
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


,AGE,SEX,CHESTPAINTYPE,RESTINGBP,CHOLESTEROL,FASTINGBS,RESTINGECG,MAXHR,EXERCISEANGINA,OLDPEAK,ST_SLOPE,HEARTDISEASE
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


Utilizando a função _info()_, é possível observar uma série de informações que serão úteis no decorrer de toda a análise

Possuímos no total 12 _features_, sendo elas:

- 5 variáveis categóricas

- 6 variáveis numéricas

- 1 target

<b>Não foi identificado nenhum valor nulo a ser tratado</b>

### Verificando Balenceamento da <b>Classe</b> alvo para modelagem

In [125]:
balance = df['HEARTDISEASE'].value_counts(normalize=True).round(2)
balance.index = balance.index.map({0:'Paciente Saúdavel', 1:'Parada Cardíaca'})
balance * 100

Parada Cardíaca      55.0
Paciente Saúdavel    45.0
Name: HEARTDISEASE, dtype: float64

Como podemos observar, as classes estão balanceadas, 

não necessitando de tratamentos como: <b>_Oversampling_</b> ou <b>_Undersampling_</b>


### Visão descritiva

Podemos utilizar a função _describe()_ para termos uma maior visão analítica dos dados

Como, por exemplo, os valores **mínimo**, **máximo** e os **percentis** de cada variável

In [96]:
df.describe()

,AGE,RESTINGBP,CHOLESTEROL,FASTINGBS,MAXHR,OLDPEAK,HEARTDISEASE
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


## Criação do Pipeline

Aqui, selecionamos as variáveis numéricas e categóricas

Esta parte será útil para Categorizar as <b>_Features_</b> dentro do <b>_Pipeline_</b> que será criado

In [105]:
cat_feat = df.select_dtypes(np.object_).columns.tolist()

num_feat = df.select_dtypes(np.number).columns.tolist()

Aqui será criado o _Pipeline_ que conterá os passos que serão utilizados para o <b>pré-processamento</b> dos dados

In [109]:
cat_feat = df.select_dtypes(np.object_).columns.tolist()
preprocess=Pipeline([
    ('ct', ColumnTransformer([
        ('onehot', OneHotEncoder(), cat_feat)     
    ],
    remainder='passthrough'),
    )
])

A partir do _Pipeline_ criado, podemos continuar a utilizar os dados na forma bruta para as análises.

Isso, em alguns casos, pode ser benéfico, pois o tratamento será executado quando for chamado, junto à etapa de modelagem

In [126]:
import plotly.express as px
plot_ = df['HEARTDISEASE'].value_counts().reset_index()
fig = px.bar(plot_, 
        x='index', 
        y='HEARTDISEASE',
        labels={'index':'Classes', 'HEARTDISEASE':''},
        title='Balanceamente de Classes Preditoras',
        width=600)
fig.update_xaxes(type='category')
fig.show()
